In [166]:
import pandas as pd
import os
from datetime import datetime
import copy

In [167]:
os.getcwd()

'C:\\Users\\shrey\\OneDrive - UW-Madison\\UW\\field-paper\\code\\genData'

In [168]:
################# GLOBAL VARIABLES ######################

# Definitions from PLFS 2018-19 Annual Report, Concepts and Definitions (2.38.1)
# Problematic codes for self employed criteria: `21` - worked in household enterprises (self-employed) as helper
SELF_EMP_CODES = ["11", "12", "21"]
REG_EMP_CODES = ["31"]
CASUAL_EMP_CODES = ["41", "42", "51", "61", "62", "71", "72"]

NOT_REG_CODES = SELF_EMP_CODES + CASUAL_EMP_CODES

EMP_CODES = SELF_EMP_CODES + REG_EMP_CODES + CASUAL_EMP_CODES
UNEMP_CODES = ["81", "82"]
LF_CODES = EMP_CODES + UNEMP_CODES
NOT_IN_LF_CODES = [str(x) for x in list(range(91,100))]

### Name of the game: 
1) Write a cleaner version of the code from before, 
    1) Get average time spent in home production, leisure (and work?). Group by work status
    2) See difference across north/south for couples where both agents are working.
    3) In order to get proper subsetting, I need to make match husbands and wives, just like I did in PLFS, so that I can subset married women with 15-49 age.
    
In order to this, it is important to first write code that gets me a data frame. This is because reading from the text file is costly. 

In [169]:
# Level 1: Get start bits, end bits and name of variable
# This Level contains States -- not exactly but still..
layoutFile_L1 = pd.read_excel("../../raw/TUS_2019/Data_Layout_TUS106.xlsx", header=7,nrows=30,usecols=[1,6,8])
layoutFile_L1.columns=['item', 'start', 'end']
layoutFile_L1.set_index("item", inplace=True)

In [170]:
# Read the corresponding text file
L1_txt = open("../../raw/TUS_2019/TUS106_L01.TXT")
start = layoutFile_L1['start']
end = layoutFile_L1['end']
rows = []
for line in L1_txt:
    line = line.strip()
    row = [line[start[i]-1:end[i]].strip() for i in range(len(layoutFile_L1))]
    rows.append(row)

In [171]:
df_L1 = pd.DataFrame(rows, columns=layoutFile_L1.index)
df_L1.head()

item,Schedule ID,FSU Serial No.,Schedule,survey year,Sector,NSS-Region,District,Stratum,Sub-Stratum,Sub-Round,...,Date of Despatch,Time to canvass(minutes),No. of investigators(FI/ASO) in the team,Remarks in block 7,Remarks in block 8,Remarks elsewhere in Sch.,Remarks elsewhere in Sch.,Blank,NSC,MULT
0,TUS,10202,106,2019,1,011,19,01,31,1,...,130219,90,1,2,2,2,2,,4,1281550
1,TUS,10202,106,2019,1,011,19,01,31,1,...,130219,90,1,2,2,2,2,,4,1281550
2,TUS,10202,106,2019,1,011,19,01,31,1,...,130219,90,1,2,2,2,2,,4,1281550
3,TUS,10202,106,2019,1,011,19,01,31,1,...,130219,90,1,2,2,2,2,,4,1281550
4,TUS,10202,106,2019,1,011,19,01,31,1,...,130219,90,1,2,2,2,2,,4,1281550


It looks like the dataset does not have State Codes/Names. But it does have NSS-Region. And it looks like that the first two digits of the NSS-Region Code is actually State Codes. I am quite sure that these state codes will be the same as those in PLFS.

In [172]:
df_L1["State"] = df_L1.loc[:,"NSS-Region"].apply(lambda x: x[:-1])

In [173]:
df_L1["north"] = pd.Series(None)
df_L1.loc[df_L1["State"].isin(["03","06","08","09","23","24"]),"north"] = 1
df_L1.loc[df_L1["State"].isin(["27","28","29","32","33","36"]),"north"] = 0

In [174]:
# Generate a Common-ID to merge across data frames. This Common-ID is a HH Level ID.
df_L1["Common-ID"] = df_L1["Schedule ID"] + df_L1["FSU Serial No."] + df_L1["Schedule"] + df_L1["survey year"] +\
                     df_L1["Sector"] + df_L1["NSS-Region"] + df_L1["District"] + df_L1["Stratum"] + df_L1["Sub-Stratum"] + \
                     df_L1["Sub-Round"] + df_L1["FOD Sub-Region"] + df_L1["Sample hhld. No."]

In [175]:
df_L1.columns

Index(['Schedule ID', 'FSU Serial No.', 'Schedule', 'survey year', 'Sector',
       'NSS-Region', 'District', 'Stratum', 'Sub-Stratum', 'Sub-Round',
       'FOD Sub-Region', 'Sample hhld. No.', 'Level ', 'Filler',
       'Informant Sl.No.', 'Response Code', 'Survey Code',
       'Substitution Code/ Casualty code', 'Filler', 'Date of Survey',
       'Date of Despatch', 'Time to canvass(minutes)',
       'No. of investigators(FI/ASO) in the team', 'Remarks in block 7',
       'Remarks in block 8', 'Remarks elsewhere in Sch.',
       'Remarks elsewhere in Sch.', 'Blank', 'NSC', 'MULT', 'State', 'north',
       'Common-ID'],
      dtype='object', name='item')

In [176]:
df_L1["Response Code"].value_counts()

Response Code
1    130123
2      6923
3      1204
4       448
9       101
Name: count, dtype: int64

**Note:** Somehow `Response Code != 1` have valid responses in L2 which is weird. But I think it is prudent to drop entries that have `Response Code != 1`. In fact save them in a `pkl` file and drop these from other files as well

```response code: informant: co-operative and capable -1, co-operative but not capable -2, busy -3,
reluctant -4, others -9.```

In [177]:
dropids = df_L1[df_L1["Response Code"] != "1"]["Common-ID"]
dropids.to_pickle("../../proc/NonResponseCommonIDs.pkl")

In [178]:
df_L1 = df_L1[df_L1["Response Code"] == "1"]

In [179]:
df_L1[[ "Response Code", "Survey Code"]].value_counts()

Response Code  Survey Code
1              1              122284
               2                7839
Name: count, dtype: int64

In [180]:
# Drop useless columns
df_L1.drop(columns=['Schedule ID', 'FSU Serial No.', 'Schedule', 'survey year',
       'NSS-Region','Stratum', 'Sub-Stratum', 'Sub-Round',
       'FOD Sub-Region', 'Sample hhld. No.', 'Level ', 'Filler',
       'Informant Sl.No.', 'Response Code', 'Survey Code',
       'Substitution Code/ Casualty code', 'Filler', 'Date of Survey',
       'Date of Despatch', 'Time to canvass(minutes)',
       'No. of investigators(FI/ASO) in the team', 'Remarks in block 7',
       'Remarks in block 8', 'Remarks elsewhere in Sch.',
       'Remarks elsewhere in Sch.', 'Blank'], inplace=True)

In [181]:
df_weights = df_L1[["Common-ID", "NSC", "MULT"]]
df_weights.to_pickle("../../proc/HHWeights.pkl")

In [182]:
df_L1.drop(columns=['NSC', "MULT"], inplace=True)

In [183]:
df_L1.columns

Index(['Sector', 'District', 'State', 'north', 'Common-ID'], dtype='object', name='item')

In [184]:
# In L2, I find out that Common-ID may have an err'd space. Checking the same for L1
df_L1["Common-ID"].apply(lambda x: len(x.replace(" ", ""))).value_counts()

Common-ID
32    121985
31      8138
Name: count, dtype: int64

In [185]:
df_L1["Common-ID"] = df_L1["Common-ID"].apply(lambda x: x.replace(" ", ""))

Drop the ones that have length less than 32

In [201]:
df_L1 = df_L1[df_L1["Common-ID"].apply(lambda x: len(x)) == 32]

In [186]:
# Level 2: Contains marital status and gender. 
layoutFile_L2 = pd.read_excel("../../raw/TUS_2019/Data_Layout_TUS106.xlsx", header=43,nrows=14,usecols=[1,6,8])
layoutFile_L2.columns=['item', 'start', 'end']
layoutFile_L2.set_index("item", inplace=True)

In [187]:
L2_txt = open("../../raw/TUS_2019/TUS106_L02.TXT")
n = 0
start = layoutFile_L2['start']
end = layoutFile_L2['end']
rows = []
for line in L2_txt:
    line = line.strip()
    row = [line[start[i]-1:end[i]].strip() for i in range(len(layoutFile_L2))]
    rows.append(row)

In [188]:
df_L2 = pd.DataFrame(rows, columns=layoutFile_L2.index)
df_L2.head()

item,Common-ID,Level,Filler,Person serial no.,Relation to head,Gender,Age,marital status,highest level of education,usual principal activity: status (code),industry of work: 2-digit of NIC 2008,Blank,NSC,MULT
0,TUS10202106201910111901311011001,02,00,001,1,1,44,2,04,31,20,,4,1281550
1,TUS10202106201910111901311011001,02,00,002,2,2,32,2,11,31,85,,4,1281550
2,TUS10202106201910111901311011001,02,00,003,5,1,5,1,02,,,,4,1281550
3,TUS10202106201910111901311011001,02,00,004,5,2,3,1,01,,,,4,1281550
4,TUS10202106201910111901311011002,02,00,001,1,1,38,2,04,11,01,,4,1281550


In [189]:
df_L2.columns

Index(['Common-ID', 'Level', 'Filler', 'Person serial no.', 'Relation to head',
       'Gender', 'Age', 'marital status', 'highest level of education',
       'usual principal activity: status (code)',
       'industry of work: 2-digit of NIC 2008', 'Blank', 'NSC', 'MULT'],
      dtype='object', name='item')

In [190]:
df_L2["primID"] = df_L2["Common-ID"]+df_L2["Person serial no."]

In [191]:
df_L2.drop(columns=['Level', 'Filler', 'Blank'], inplace=True)

In [192]:
df_weights = df_L2[['Common-ID', 'Person serial no.', "primID", 'NSC', 'MULT']]
df_weights.to_pickle("../../proc/IndWeights.pkl")
df_L2.drop(columns = ['NSC', 'MULT'], inplace=True)

In [193]:
df_L2.columns

Index(['Common-ID', 'Person serial no.', 'Relation to head', 'Gender', 'Age',
       'marital status', 'highest level of education',
       'usual principal activity: status (code)',
       'industry of work: 2-digit of NIC 2008', 'primID'],
      dtype='object', name='item')

In [194]:
df_L2["Common-ID"].apply(lambda x: len(x.replace(" ", ""))).value_counts()

Common-ID
32    487826
31     30918
Name: count, dtype: int64

Looks like some Common-IDs in L2 have missing bits. About 6%. Drop them

In [202]:
df_L2["Common-ID"] = df_L2["Common-ID"].apply(lambda x: x.replace(" ", ""))
df_L2 = df_L2[df_L2["Common-ID"].apply(lambda x: len(x)) == 32]

In [203]:
# Merge all the dataframes into one and save as pickle file
df_merged = pd.merge(left=df_L1, right=df_L2, on="Common-ID", how="outer",indicator=True)

In [204]:
df_merged["_merge"].value_counts()

_merge
both          455448
right_only     32378
left_only          0
Name: count, dtype: int64

`_merge == right_only` because I drop those HHs with `Response Code ~= 1` 

In [206]:
respcodeids = pd.read_pickle("../../proc/NonResponseCommonIDs.pkl")
sum(df_merged[df_merged["_merge"] == "right_only"]["Common-ID"].isin(respcodeids))

32378

In [207]:
df_merged.columns

Index(['Sector', 'District', 'State', 'north', 'Common-ID',
       'Person serial no.', 'Relation to head', 'Gender', 'Age',
       'marital status', 'highest level of education',
       'usual principal activity: status (code)',
       'industry of work: 2-digit of NIC 2008', 'primID', '_merge'],
      dtype='object', name='item')

In [208]:
df_merged = df_merged[df_merged["_merge"] == "both"]

In [209]:
df_merged.drop(columns=["_merge"],inplace=True)

C:\Users\shrey\AppData\Local\Temp\ipykernel_18208\3764088376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged.drop(columns=["_merge"],inplace=True)


In [210]:
df_merged.to_pickle("../../proc/df_L1L2.pkl")

In [211]:
####################### END DATA GENERATION ########################